In [ ]:
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as feature

%matplotlib inline

In [ ]:
land_50m = feature.NaturalEarthFeature('physical', 'land', '50m',
                                        edgecolor='black',
                                        facecolor='#DFD1AF',
                                        linewidth=0.2)

This script 